In [1]:
#@title Run this to implement tic-tac-toe!
from copy import copy, deepcopy

def available_moves(board):
  availableMoves = []
  for r in range(len(board)):
    for c in range(len(board[r])):
      if board[r][c] == '_':
        availableMoves.append((r,c))
  return availableMoves 

def is_move_ok(loc, board):
  return loc in available_moves(board)

def check_victory(board):
  for r in board:
    if r[0] == r[1] == r[2] and r[0] != '_':
      return r[0]
  
  for c in range(len(board[0])):
    if board[0][c] == board[1][c] == board[2][c] and board[0][c] != '_':
      return board[0][c]
  
  if board[0][0] == board[1][1] == board[2][2] and board[0][0] != '_':
    return board[0][0]
  
  if board[0][2] == board[1][1] == board[2][0] and board[0][2] != '_':
    return board[0][2]
  
  if len(available_moves(board)) == 0:
    return 'draw'
  
  return 'ongoing'

def print_board(board):
  print('| ' + board[0][0] + ' | ' + board[0][1] + ' | ' + board[0][2] + ' |')
  print('| ' + board[1][0] + ' | ' + board[1][1] + ' | ' + board[1][2] + ' |')
  print('| ' + board[2][0] + ' | ' + board[2][1] + ' | ' + board[2][2] + ' |')

def play_game(player1='human', player2='human'):
  game_board = [['_', '_', '_'],
               ['_', '_', '_'],
               ['_', '_', '_']]
  victory = 'ongoing'
  player = 'X'
  while victory =='ongoing':
    print_board(game_board)
    if (player == 'X' and player1 == 'human') or (player == 'O' and player2 == 'human'):
      unparsed_coordinates = input("Where do you want to place? ") 
      try: 
        parsed = unparsed_coordinates.replace('(', "").replace(')', "") # make sure input format is right
        if ', ' in parsed:
          coords = tuple(parsed.split(", "))
        else:
          coords = tuple(parsed.split(","))
        coordinates = (int(coords[0]), int(coords[1]))
    
      except: # if people put in anything that doesn't fit the format, it will be caught here
        print("Please enter a coordinate in the correct format.")
        continue
    else:
      print("Computer", player, "is deciding on a move!")
      if player == 'X':
        coordinates = player1(game_board, player)
        if type(coordinates[1])!=int:
          coordinates = coordinates[1]
      else:
        coordinates = player2(game_board, player)
        if type(coordinates[1])!=int:
          coordinates = coordinates[1]
    if not is_move_ok(coordinates, game_board):
      print("Please enter a coordinate in bounds that is open.")
      continue
    else:
      game_board[coordinates[0]][coordinates[1]] = player
      if player == 'X':
        player = 'O'
      else:
        player = 'X'
      victory = check_victory(game_board)
  
  print_board(game_board)
  if victory == 'draw':
    print("The game ended in a draw!")
  else:
    print("Player", victory, "won the game!")

def get_board_score(board, player):
  result = check_victory(board)
  if result == 'draw' or result == 'ongoing':
    return 0
  
  elif result != player:
    return -10
  
  else:
    return 10

def other_player(player):
  if player == 'O':
      otherPlayer = 'X'
  else:
    otherPlayer = 'O'
  return otherPlayer

get_board_score(board, player) - Given a game state and player, this function returns 0 if the game is ongoing/drawn, -10 if the player has lost, or 10 if the player has won.

check_victory(board) - Given a game state, this function checks if X has won, O has won, the game has drawn, or the game is ongoing.

available_moves(board) - Given a game state, this function returns all available moves that the next player can take.

other_player(player) - Given a player, this function return the opponent player.

play_game() - This function simulates a game of Tic-Tac-Toe.

In [2]:
def maximize_score(board, player):
  if check_victory(board) != 'ongoing':
    return get_board_score(board,player)
  maximum = -float('inf') #general way to make sure that this value is less than 
                          #all possible "values" given for a game state
  moves = available_moves(board) #finds all the available positions that the 
                                 #next player can play
  for move in moves: #loops through all these available positions
    child = deepcopy(board) #deepcopy makes a copy of all elements of the 
                            #original list, while copy references all elements 
                            #of the original list
    child[move[0]][move[1]] = player #assigns the board position to the symbol 
                                     #that the player chose
    maximize = minimize_score(child, other_player(player)) #minimize the score 
                                                           #of the other player; 
                                                           #we will create a 
                                                           #minimize function 
    maximum = max(maximize,maximum) #maximize the minimum score of the other 
                                    #player
  return maximum 

In [3]:
def minimize_score(board, player):
  if check_victory(board) != 'ongoing':
    return get_board_score(board,other_player(player))
  minimum = float('inf')
  moves = available_moves(board)
  for move in moves:
    child = deepcopy(board)
    child[move[0]][move[1]] = player
    minimize = maximize_score(child, other_player(player))
    minimum = min(minimize,minimum)
  return minimum 

In [4]:
def minimax_move(board, player):
  best_move = None #we will return this at the end of the function
  value = -float('inf') #will track the maximum lowest score for the other 
                        #player
  moves = available_moves(board)
  for move in moves:
    child = deepcopy(board)
    child[move[0]][move[1]] = player
    childVal = minimize_score(child, other_player(player)) #minimize the score 
                                                           #of the other player
    if value <= childVal: #this means that the maximum lowest score for the 
                          #child is greater than the previous maximum lowest 
                          #score calculated
      value = childVal #change value variable and best move
      best_move = move
  return best_move #finally return best move

In [6]:
play_game(player1=minimax_move, player2='human')

| _ | _ | _ |
| _ | _ | _ |
| _ | _ | _ |
Computer X is deciding on a move!
| _ | _ | _ |
| _ | _ | _ |
| _ | _ | X |
Where do you want to place? 2,1
| _ | _ | _ |
| _ | _ | _ |
| _ | O | X |
Computer X is deciding on a move!
| _ | _ | _ |
| _ | _ | X |
| _ | O | X |
Where do you want to place? 0,2
| _ | _ | O |
| _ | _ | X |
| _ | O | X |
Computer X is deciding on a move!
| _ | _ | O |
| _ | X | X |
| _ | O | X |
Where do you want to place? 1,0
| _ | _ | O |
| O | X | X |
| _ | O | X |
Computer X is deciding on a move!
| X | _ | O |
| O | X | X |
| _ | O | X |
Player X won the game!
